In [1]:
# Código de conexão com API baseado no ótimo artigo do Gabriel Luz (Data Hackers):
# https://medium.com/data-hackers/an%C3%A1lise-de-dados-de-uma-playlist-do-spotify-com-python-e-power-bi-b3149c2bd633

#### 1º Case de Estudo - Buscando padrões de gosto nas minhas playlists do Spotify

In [13]:
import sys

import json
import pandas as pd

import spotipy
import spotipy.util as util

In [17]:
# id e secrets da api do spotify, cadastradas no site 'spotify developers'
client_id = '535eb94db1a944e2a5ac5df95bbff061'
client_secret = '40b6b65222014e6ea627b99e45f66171'
redirect_uri = 'http://www.google.com'

# username da conta, tipo de permissão que será abilitada pelo dono da conta
username = 'alexeifloyd'
scope = 'user-library-read playlist-read-private'

In [18]:
# Guarda em uma variável token todas as informações acima, gerando um pedido de autenticação

try:
    token = util.prompt_for_user_token(username, scope, client_id=client_id, client_secret=client_secret, redirect_uri=redirect_uri)
    sp=spotipy.Spotify(auth= token)
except:
    print('Token is not accesible for ' + username)



            User authentication requires interaction with your
            web browser. Once you enter your credentials and
            give authorization, you will be redirected to
            a url.  Paste that url you were directed to to
            complete the authorization.

        
Opened https://accounts.spotify.com/authorize?client_id=535eb94db1a944e2a5ac5df95bbff061&response_type=code&redirect_uri=http%3A%2F%2Fwww.google.com&scope=playlist-read-private+user-library-read in your browser


Enter the URL you were redirected to: https://www.google.com/?code=AQBijm_RDmBZ-QZ0kwNtFnqDi6PVL3lIyW4lep-9txs-paEWuYFyBo2UzYIZrdAubgUOhKY1y39uDDpfvXYdL6P89hY4h9E5BEY6RV_nBGYNE91S8tFVdIiNwxjEnsa3ZpeclJXq19KRESgOdozFYJH4MwZmXYObqHKxfxDLJx_5IY9Q0fB0uCdtvu8duCTAuamvUC0ZNDoKo6Xoa4ABll-2TJj6o8FXbcW4M8uG-EpbrEKJkHCi




#### Capturando as informações de uma playlist e as features correspondentes

In [19]:
def get_playlist_content(username, play_id, sp):
    offset = 0
    songs = []
    while True:
        content = sp.user_playlist_tracks(username, play_id, fields=None, limit=100, offset=offset, market=None)
        songs += content['items']
        if content['next'] is not None:
            offset += 100
        else:
            break

    with open('{}-{}.json'.format(username, play_id), 'w') as outfile:
        json.dump(songs, outfile)

In [20]:
def get_playlist_audio_features(username, play_id, sp):
    offset = 0
    songs = []
    items = []
    ids = []
    while True:
        content = sp.user_playlist_tracks(username, play_id, fields=None, limit=100, offset=offset, market=None)
        songs += content['items']
        if content['next'] is not None:
            offset += 100
        else:
            break

    for i in songs:
        ids.append(i['track']['id'])

    index = 0
    audio_features = []
    while index < len(ids):
        audio_features += sp.audio_features(ids[index:index + 50])
        index += 50

    features_list = []
    for features in audio_features:
        features_list.append([features['energy'], features['liveness'],
                              features['tempo'], features['speechiness'],
                              features['acousticness'], features['instrumentalness'],
                              features['time_signature'], features['danceability'],
                              features['key'], features['duration_ms'],
                              features['loudness'], features['valence'],
                              features['mode'], features['type'],
                              features['uri']])

    df = pd.DataFrame(features_list, columns=['energy', 'liveness', 'tempo', 'speechiness', 'acousticness', 
                                              'instrumentalness', 'time_signature', 'danceability', 'key',
                                              'duration_ms', 'loudness', 'valence', 'mode', 'type', 'uri'])
    df.to_csv('{}-{}.csv'.format(username, play_id), index=False)

In [21]:
def get_user_playlist(username, sp):
        playlists = sp.user_playlists(username)
        for playlist in playlists['items']:
            print("Name: {}, Number of songs: {}, Playlist ID: {} ".
                  format(playlist['name'].encode('utf8'), playlist['tracks']['total'], playlist['id']))
            play_id = playlist['id']
            print("Getting playlist content")
            get_playlist_content(username, play_id, sp)
            print("Getting playlist audio features")
            get_playlist_audio_features(username, play_id, sp)

In [22]:
if token:
    
    sp = spotipy.Spotify(auth=token)
    
    print("Getting user playlist")
    get_user_playlist(username, sp)
    
else:
    print("Can't get token for", username)

Getting user playlist
Name: b'Na Hora H', Number of songs: 11, Playlist ID: 0p2qr9FmaUPvveDs2NdxbE 
Getting playlist content
Getting playlist audio features
Name: b'Psytrance Supernova', Number of songs: 70, Playlist ID: 37i9dQZF1DWXWbLEOaHnU3 
Getting playlist content
Getting playlist audio features
Name: b'Mary Elizabeth McGlynn - Silent Hill \xf0\x9f\x8c\xab\xf0\x9f\x92\xa7\xe2\x9d\x84\xef\xb8\x8f', Number of songs: 16, Playlist ID: 7M1ENInXOBhepegWFdfLw4 
Getting playlist content
Getting playlist audio features
Name: b'M\xc3\xbasicas de casamento - O Amor \xc3\xa9 Simples', Number of songs: 64, Playlist ID: 4EmIhy7DuTKFIxgujxW4S4 
Getting playlist content
Getting playlist audio features
Name: b'Dark Souls', Number of songs: 18, Playlist ID: 0HyomGh26ecF1V2tRQ4iIC 
Getting playlist content
Getting playlist audio features
Name: b'Curtida no R\xc3\xa1dio', Number of songs: 31, Playlist ID: 52IXURukhNfGA3cyyNIOPW 
Getting playlist content
Getting playlist audio features
Name: b'Samurai

#### Exemplo de teste

In [ ]:
# Exemplo vindo da documentação da biblioteca

def show_tracks(tracks):
    for i, item in enumerate(tracks['items']):
        track = item['track']
        print("   %d %24.24s %s" % (i, track['artists'][0]['name'], track['name']))

if token:
    sp = spotipy.Spotify(auth=token)
    playlistss = sp.user_playlists(username)
    for playlist in playlistss['items']:
        if playlist['owner']['id'] == username:
            print()
            print(playlist['name'])
            print('  Total Tracks:', playlist['tracks']['total'])
            results = sp.user_playlist(username, playlist['id'],fields="tracks,next")
            tracks = results['tracks']
            show_tracks(tracks)
            while tracks['next']:
                tracks = sp.next(tracks)
                show_tracks(tracks)
else:
    print("Can't get token for", username)